In [82]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pylab
pylab.rcParams['figure.figsize'] = (12,8)  # set default plot size

# make dataframes not wrap
pd.set_option('expand_frame_repr', False)

from collections import Counter
from dtutil.sqltools import query_to_dataframe, list_to_query_list
from dtutil.util import init_cdb, run_cdb_query

In [83]:
mssql_conn = init_cdb()

# Looking for WAN usage of non WAN licenses

In this analysis we will look at customers that are using non-WAN licenses in multiple countries. This is a somewhat approximate analysis because our ability to detect location is only based on the IP address we get and with things like proxies and VPNs they are not a perfect locator of a user.  Some other caveats in this analysis are:

* We currently only map IP addresses to countries so we cannot look for WAN usage in countries with multiple time zones
* This analysis looks at all starts since 2017-07-01 (about 7.5 months)

## Collecting the data

The first step is to find all customers that have use in multiple countries.

In [84]:
query = """
with cte as (
select distinct c.cust_code, s.country
from lds_starts as s with(nolock)
	inner join nxscontr as c on s.serial_num = c.contract_num
where s.log_date > '2017-07-01'
	and isnumeric(s.serial_num)=1 and s.serial_num between 30000 and 80000
)
select cust_code from cte
group by cust_code
having count(cust_code) > 1
"""
custids = run_cdb_query(query)

We want to remove from this the set of known WAN customers

In [85]:
set_of_custids = {x[0] for x in custids}
known_wan_customers = {208, 541, 551, 950, 1447, 3808, 3916, 4840, 6571, 6802, 7480, 8568, 8743, 8813, 8766, 9244,
                       9532, 10081, 10137, 10244, 10484, 10653, 11010, 11489, 11800, 10060015}
of_interest = set_of_custids - known_wan_customers

Next, we want to collect the start data by serial number so that we can see how many starts are in each country.  We don't really care about some occassional use in other countries (when engineers are traveling). In this analysis what we want to focus on is significant use in multiple countries.

In [86]:
query = """
select distinct c.cust_code, s.serial_num, s.country, count(s.id), a.Name
from lds_starts as s with(nolock)
	inner join nxscontr as c
        on s.serial_num = c.contract_num
    inner join Applied_Wave_Research_MSCRM.dbo.AccountBase as a
		on c.cust_code = a.AccountNumber
where s.log_date > '2017-07-01'
	and isnumeric(s.serial_num)=1 and c.cust_code in ({})
group by c.cust_code, s.serial_num, s.country, a.Name
""".format(','.join([str(x) for x in of_interest]))
data = run_cdb_query(query)

We need to organize this data as well as filter out the countries where there are only a few uses. For now we will eliminate all countries that have less than **10 starts**.

In [87]:
class CustomerCounter:
    def __init__(self, name, custid):
        self.name = name
        self.custid = custid
        self.sn_set = set()
        self.starts = Counter()
    def __str__(self):
        return '{}({})'.format(self.custid, self.starts)
    def __repr__(self):
        return '{}({})'.format(self.custid, self.starts)

by_custid = {}
for custid, sn_str, country, count, name in data:
    if custid in by_custid:
        obj = by_custid[custid]
    else:
        obj = CustomerCounter(name, custid)
    obj.starts[country] += count
    obj.sn_set.add(sn_str)
    by_custid[custid] = obj

# next eliminate the countries with only a few starts
to_delete = []
for custid, cc_obj in by_custid.items():
    num_significant_countries = 0
    for country, starts in cc_obj.starts.items():
        if starts > 10:
            num_significant_countries += 1
    if num_significant_countries < 2:
        to_delete.append(custid)
print('Starting with {} custids'.format(len(by_custid)))
for custid in to_delete:
    del by_custid[custid]
print('Finished with {} custids'.format(len(by_custid)))

Starting with 104 custids
Finished with 34 custids


Now that we have pruned the list we can print it out.

## Results - Caveats

I have not filtered out all the customers that have WAN licenses. In addition, some customers have licenses such as WAN4 and only the salesperson know what countries are intended in that use.

It must be remembered that not all use below is improper and the salesperson in the territory will be able to know if the use at a customer actually violates the intent of their licenses. We must also remember that customers traveling are allowed to use their license so before we could contact any customer on this we must make sure that different users are actually using the software in different countries.

**Please contact Dane to verify the data on a customer and discuss the situation with sales management prior to contacting any customer regarding this data**

In [88]:
by_custid[10369].comment = 'US user accounts for use in JP'
by_custid[3602].comment = 'Initial research indicates a problem'
by_custid[19].comment = 'All use appears to be traveling US users'
by_custid[11286].comment = 'Initial research indicates a problem'
by_custid[11086].comment = 'Initial research indicates a problem'
by_custid[11572].comment = 'All locked licenses'
by_custid[566].comment = 'One user on travel'
by_custid[8756].comment = 'All locked licenses'
by_custid[2860].comment = 'One user on travel'
by_custid[10784].comment = 'All locked licenses'
by_custid[10716].comment = 'All locked licenses'
by_custid[1520].comment = 'One user on travel'
by_custid[11747].comment = 'One user on travel'
by_custid[11508].comment = 'Initial research indicates a problem'
by_custid[10030073].comment = 'Initial research indicates a problem'
by_custid[11185].comment = 'Research indicates floating to India'
by_custid[9158].comment = 'Research indicates floating to GB'
by_custid[10783].comment = 'So much DE usage seems strange, possible network config issue?'
by_custid[6706].comment = 'Most use appears to be traveling users'
by_custid[6464].comment = 'Use appears to be traveling users'
by_custid[8819].comment = 'Initial research indicates a problem but off maint customer'
by_custid[11645].comment = 'Use appears to be traveling users'
by_custid[7053].comment = 'Research indicates floating to Canada'
by_custid[8659].comment = 'Use appears to be traveling users'
by_custid[10243].comment = 'Use appears to be traveling users'
by_custid[9121].comment = 'Use appears to be traveling users'
by_custid[941].comment = 'Use appears to be traveling users'
by_custid[11719].comment = 'All locked licenses'
by_custid[10230].comment = 'Use appears to be traveling users'
by_custid[312].comment = 'Use appears to be traveling users'


for obj in by_custid.values():
    print('{} - {}'.format(obj.custid, obj.name))
    for k, v in obj.starts.items():
        if v > 10:
            print('    {} - {}'.format(k, v))
    if hasattr(obj, 'comment'):
        print('    ====== {}'.format(getattr(obj, 'comment')))
    print()

10502 - Qorvo - GreanPeak BV Netherlands
    HK - 14
    NL - 68

3602 - SAAB Grintek Defence (PTY) Ltd.
    ZA - 948
    SE - 398
    ====== Initial research indicates a problem

10243 - Huawei Technologies Co., Ltd. - Shenzhen
    CN - 10736
    SE - 23
    ====== Use appears to be traveling users

11286 - Huawei Technologies Dusseldorf GmbH - European Research Center
    DE - 50
    GB - 67
    ====== Initial research indicates a problem

10783 - Qualcomm Technologies, Inc. - Maitland
    DE - 19392
    ES - 23
    US - 183
    ====== So much DE usage seems strange, possible network config issue?

10784 - Agile Microwave Technology Inc
    EE - 42
    FI - 53
    US - 119
    ====== All locked licenses

11301 - Qualcomm Technologies Netherlands BV
    EU - 537
    NL - 38

2860 - European Space Agency (ESA) - European Space Research and Technology Centre (ESTEC)
    ES - 21
    NL - 477
    ====== One user on travel

6706 - Qorvo - TQ Munich
    FR - 20
    DE - 14763
    US - 529
 

## Details for a company

In [ ]:
cust_of_interest = 10484
query = """
select distinct user_name, machine_name, country, count(id)
from lds_starts with(nolock)
where log_date > '2017-07-01'
and serial_num in {}
group by user_name, machine_name, country
order by user_name, machine_name, country
""".format(list_to_query_list(by_custid[cust_of_interest].sn_set))
query_to_dataframe(mssql_conn.cursor(),query)